# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os

import sys
sys.path.insert(1, './../src/')

import time 

import load_option_data_01 

from pathlib import Path

import bsm_pricer as bsm
import config
import datetime
import level_1_filters as f1
import level_2_filters as f2
import level_3_filters as f3
import load_option_data_01 
import load_option_data_01 as l1


import time 
import warnings
import wrds

from scipy.stats import norm
from scipy.spatial.distance import cdist

In [2]:
import importlib

In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
warnings.filterwarnings("ignore")

In [4]:
OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME

START_DATE_01 =config.START_DATE_01
END_DATE_01 = config.END_DATE_01

START_DATE_02 =config.START_DATE_02
END_DATE_02 = config.END_DATE_02

NOTE_START = START_DATE_01
NOTE_END = END_DATE_01

In [5]:
DATE_RANGE =f'{pd.Timestamp(START_DATE_01):%Y-%m}_{pd.Timestamp(END_DATE_02):%Y-%m}'

# Functions

In [6]:
# --- Black-Scholes elasticity ---
def bs_elasticity(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T))
        delta = norm.cdf(d1)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)) - S * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
    return (delta * S / price), price

# --- Kernel Weights ---
def kernel_weights(moneyness_grid, maturity_grid, k_s, ttm, bw_m=0.0125, bw_t=10):
    grid = np.column_stack((moneyness_grid, maturity_grid))
    point = np.array([[k_s, ttm]])
    dists = cdist(grid, point, 'euclidean')[:, 0]
    weights = np.exp(-0.5 * (dists / np.array([bw_m, bw_t])).sum())
    return weights / weights.sum()

# --- Construct a single day portfolio ---
def construct_portfolio(data, k_s_target, ttm_target, option_type='call', r=0.01):
    subset = data[(data['option_type'] == option_type)]
    weights = kernel_weights(subset['moneyness'], subset['ttm'], k_s_target, ttm_target)
    subset = subset.assign(weight=weights)
    subset = subset[subset['weight'] > 0.01]
    subset['weight'] /= subset['weight'].sum()

    # Leverage-adjusted returns
    elast, price = bs_elasticity(
        S=subset['underlying'], K=subset['strike'], T=subset['ttm']/365,
        r=r, sigma=subset['iv'], option_type=option_type
    )
    subset['leverage_return'] = subset['daily_return'] / elast

    return (subset['leverage_return'] * subset['weight']).sum()

# --- Main Loop (simplified) ---
def build_portfolios(option_data, m_grid, ttm_grid, option_types=['call', 'put']):
    portfolios = []
    for opt_type in option_types:
        for k_s in m_grid:
            for ttm in ttm_grid:
                ret = construct_portfolio(option_data, k_s, ttm, option_type=opt_type)
                portfolios.append({
                    'type': opt_type,
                    'moneyness': k_s,
                    'ttm': ttm,
                    'return': ret
                })
    return pd.DataFrame(portfolios)


In [15]:
# read filtered data

spx_filtered = pd.read_parquet(Path(DATA_DIR / f'spx_filtered_final_{DATE_RANGE}.parquet'))
spx_filtered

secid     open    close cp_flag        IV  \
date       exdate     moneyness                                                 
1996-01-04 1996-01-20 1.036102   108105.0   621.32   617.70       C  0.100588   
                      1.019913   108105.0   621.32   617.70       C  0.097356   
                      1.028007   108105.0   621.32   617.70       C  0.101756   
                      0.987534   108105.0   621.32   617.70       C  0.082711   
           1996-02-17 0.971345   108105.0   621.32   617.70       C  0.085712   
...                                   ...      ...      ...     ...       ...   
2019-12-31 2020-06-19 1.044639   108105.0  3215.18  3230.78       P  0.117063   
                      1.052377   108105.0  3215.18  3230.78       P  0.113900   
                      1.060116   108105.0  3215.18  3230.78       P  0.110925   
                      1.067854   108105.0  3215.18  3230.78       P  0.108540   
                      1.075592   108105.0  3215.18  3230.78       P  0.106814   

                                 tb_m3  volume  open_interest  best_bid  \
date       exdate     moneyness                                           
1996-01-04 1996-01-20 1.036102    5.04     0.0         6593.0    0.1875   
                      1.019913    5.04  4022.0         5969.0    1.1875   
                      1.028007    5.04  1627.0         6224.0    0.6250   
                      0.987534    5.04   444.0         5905.0   10.0000   
           1996-02-17 0.971345    5.04     0.0          398.0   21.1250   
...                                ...     ...            ...       ...   
2019-12-31 2020-06-19 1.044639    1.52     0.0          395.0  192.2000   
                      1.052377    1.52     0.0          163.0  208.2000   
                      1.060116    1.52    20.0          310.0  222.8000   
                      1.067854    1.52     0.0           12.0  241.2000   
                      1.075592    1.52     0.0            0.0  260.9000   

                                 best_offer  strike_price  contract_size  \
date       exdate     moneyness                                            
1996-01-04 1996-01-20 1.036102        0.375         640.0          100.0   
                      1.019913        1.375         630.0          100.0   
                      1.028007        0.750         635.0          100.0   
                      0.987534       10.375         610.0          100.0   
           1996-02-17 0.971345       22.125         600.0          100.0   
...                                     ...           ...            ...   
2019-12-31 2020-06-19 1.044639      193.000        3375.0          100.0   
                      1.052377      209.100        3400.0          100.0   
                      1.060116      228.900        3425.0          100.0   
                      1.067854      247.600        3450.0          100.0   
                      1.075592      267.500        3475.0          100.0   

                                 mid_price days_to_maturity  \
date       exdate     moneyness                               
1996-01-04 1996-01-20 1.036102     0.28125          16 days   
                      1.019913     1.28125          16 days   
                      1.028007     0.68750          16 days   
                      0.987534    10.18750          16 days   
           1996-02-17 0.971345    21.62500          44 days   
...                                    ...              ...   
2019-12-31 2020-06-19 1.044639   192.60000         171 days   
                      1.052377   208.65000         171 days   
                      1.060116   225.85000         171 days   
                      1.067854   244.40000         171 days   
                      1.075592   264.20000         171 days   

                                 pc_parity_int_rate  intrinsic    log_iv  \
date       exdate     moneyness                                            
1996-01-04 1996-01-20 1.036102             0.

## Portfolio Construction Process

Portfolios are defined as follows:
- Portfolio return from $t-1$ to $t$ is defined as $\Rho_{C or P,m,T,t}$, where $\Rho$ is the daily return of a call ($C$) or put ($P$) portfolio, with moneyness $m$ and maturity $T$ days, where $T \in {30, 60, 90}$, at time $t$.
$$
\begin{align}
\Rho_{C,m,t} &= xC_t - yr_f \\
\Rho_{P,m,t} &= -zP_t + br_f

\end{align}
$$


In [16]:
spx_filtered = spx_filtered.reset_index().set_index(['date','cp_flag','moneyness', 'days_to_maturity'])

In [17]:
spx_filtered

exdate     secid     open  \
date       cp_flag moneyness days_to_maturity                                 
1996-01-04 C       1.036102  16 days          1996-01-20  108105.0   621.32   
                   1.019913  16 days          1996-01-20  108105.0   621.32   
                   1.028007  16 days          1996-01-20  108105.0   621.32   
                   0.987534  16 days          1996-01-20  108105.0   621.32   
                   0.971345  44 days          1996-02-17  108105.0   621.32   
...                                                  ...       ...      ...   
2019-12-31 P       1.044639  171 days         2020-06-19  108105.0  3215.18   
                   1.052377  171 days         2020-06-19  108105.0  3215.18   
                   1.060116  171 days         2020-06-19  108105.0  3215.18   
                   1.067854  171 days         2020-06-19  108105.0  3215.18   
                   1.075592  171 days         2020-06-19  108105.0  3215.18   

                                                 close        IV  tb_m3  \
date       cp_flag moneyness days_to_maturity                             
1996-01-04 C       1.036102  16 days            617.70  0.100588   5.04   
                   1.019913  16 days            617.70  0.097356   5.04   
                   1.028007  16 days            617.70  0.101756   5.04   
                   0.987534  16 days            617.70  0.082711   5.04   
                   0.971345  44 days            617.70  0.085712   5.04   
...                                                ...       ...    ...   
2019-12-31 P       1.044639  171 days          3230.78  0.117063   1.52   
                   1.052377  171 days          3230.78  0.113900   1.52   
                   1.060116  171 days          3230.78  0.110925   1.52   
                   1.067854  171 days          3230.78  0.108540   1.52   
                   1.075592  171 days          3230.78  0.106814   1.52   

                                               volume  open_interest  \
date       cp_flag moneyness days_to_maturity                          
1996-01-04 C       1.036102  16 days              0.0         6593.0   
                   1.019913  16 days           4022.0         5969.0   
                   1.028007  16 days           1627.0         6224.0   
                   0.987534  16 days            444.0         5905.0   
                   0.971345  44 days              0.0          398.0   
...                                               ...            ...   
2019-12-31 P       1.044639  171 days             0.0          395.0   
                   1.052377  171 days             0.0          163.0   
                   1.060116  171 days            20.0          310.0   
                   1.067854  171 days             0.0           12.0   
                   1.075592  171 days             0.0            0.0   

                                               best_bid  best_offer  \
date       cp_flag moneyness days_to_maturity                         
1996-01-04 C       1.036102  16 days             0.1875       0.375   
                   1.019913  16 days             1.1875       1.375   
                   1.028007  16 days             0.6250       0.750   
                   0.987534  16 days            10.0000      10.375   
                   0.971345  44 days            21.1250      22.125   
...                                                 ...         ...   
2019-12-31 P       1.044639  171 days          192.2000     193.000   
                   1.052377  171 days          208.2000     209.100   
                   1.060116  171 days          222.8000     228.900   
                   1.067854  171 days          241.2000     247.600   
                   1.075592  171 days          260.9000     267.500   

                                               strike_price  contract_size  \
date       cp_flag moneyness days_to_maturity                                
1996-01-04 C       1.036102  

In [8]:
pd.date_tDATE_RANGE

AttributeError: module 'pandas' has no attribute 'date_tDATE_RANGE'

In [ ]:
moneyness_tgts = [0.90, 0.925, 0.95, 0.975, 1.00, 1.025, 1.05, 1.075, 1.10] # K/S, so < 1 = puts are OTM/calls are ITM, and > 1 calls are OTM/puts are ITM
maturity_tgts = [30, 60, 90]  # in days. For HKM, these are averaged
option_types = ['C', 'P']  # 'call' and 'put' options

portfolio_idx = pd.MultiIndex.from_tuples(
    [(opt_type, k_s, ttm) for opt_type in option_types for k_s in moneyness_tgts for ttm in maturity_tgts for trade_date in spx_filtered.index.get_level_values('date')],
    names=['option_type', 'moneyness', 'days_to_maturity', 'trade_date']
)

portfolios = pd.DataFrame(index=portfolio_idx, columns=['daily_rets', 'monthly_rets'])
portfolios

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001F7B5548090>>
Traceback (most recent call last):
  File "c:\Users\viren.desai\OneDrive - Aprio, LLP\Documents\repos\constantinides_2013_options\puzzle\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
